In [41]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import random
random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from src.tradecli import *
from src.utils import *
from src.journal import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [43]:
display(full_df)

,trade_uid,type_3_m1,type_3_M,numerical_confidence,outcome,type_1_m5,type_2_m30,type_1_m1,type_3_h4,type_3_w,htf_poi_ltf_confirmation,type_2_m15,type_3_m15,type_2_h4,return,close_price,type_1_m30,management_strategy,potential_risk_reward,initial_risk_reward,entry_price,type_3_d,type_2_M,type_2_m1,type_2_w,entry_time,potential_price,potential_return,SL_distance,type_3_h1,type_1_h4,type_2_m5,type_2_h1,type_1_m15,risk_reward_ratio,tp_price,type_1_h1,type_1_d,sl_price,type_1_M,session,initial_return,type_3_m5,type_3_m30,type_2_d,type_1_w,side,TP_distance,htf_poi_ltf_confirmation_False,htf_poi_ltf_confirmation_True,management_strategy_be_after_1r,management_strategy_be_after_push,management_strategy_close_early,management_strategy_no_management,management_strategy_strategy_2,outcome_loss,outcome_win,has_type_1,has_type_2,has_type_3,has_time_frame
0,1,False,False,3.0,NaN,False,False,True,False,False,False,False,True,False,NaN,NaN,False,no_management,NaN,2.000000,1.1000,False,False,False,False,2025-01-02 01:45:55.920267,NaN,NaN,0.0050,False,False,False,False,False,2.000000,1.1100,False,False,1.0950,False,tokyo,0.0100,False,False,False,False,long,0.0100,True,False,False,False,False,True,False,False,False,True,False,True,False
1,2,False,False,2.0,loss,False,False,False,False,False,True,False,False,False,-0.0145,1.1855,False,no_management,NaN,0.666667,1.2000,False,False,False,False,2025-01-02 01:45:55.920785,NaN,NaN,0.0150,False,False,False,True,False,0.666667,1.2100,False,False,1.1850,False,tokyo,0.0100,False,False,False,False,long,0.0100,False,True,False,False,False,True,False,True,False,False,True,False,False
2,3,False,False,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,False,no_management,NaN,10.000000,1.3000,False,False,False,False,2024-12-27 10:00:01.000000,NaN,NaN,0.5000,False,False,False,False,False,10.000000,1.3300,False,False,1.2970,False,london,0.0300,False,False,False,False,long,0.0300,True,False,False,False,False,True,False,False,False,False,False,False,False
3,4,False,False,4.0,win,False,False,False,False,False,False,False,False,False,0.0050,1.3050,False,strategy_2,5.000000,10.000000,1.3000,False,False,False,False,2024-12-25 10:00:00.000000,1.3150,0.0150,0.5000,False,False,False,False,False,10.000000,1.3300,False,False,1.2970,False,london,0.0300,False,False,False,False,long,0.0300,True,False,False,False,False,False,True,False,True,False,False,False,False
4,5,False,False,5.0,win,False,False,True,False,False,False,False,False,False,0.0117,1.1495,False,no_management,5.245614,5.000000,1.1378,False,False,False,False,2025-01-02 01:45:55.923263,1.1677,0.0299,0.0057,False,False,False,True,False,5.000000,1.1663,False,False,1.1321,False,tokyo,0.0285,False,False,False,False,long,0.0285,True,False,False,False,False,True,False,False,True,True,True,False,False
5,6,False,False,2.0,win,False,False,False,False,False,False,False,False,False,0.0180,1.1511,False,no_management,1.518519,5.913580,1.1331,False,False,False,False,2025-01-02 01:45:55.923765,1.1454,0.0123,0.0081,False,False,False,False,False,5.913580,1.1810,False,False,1.1250,True,tokyo,0.0479,False,False,True,False,long,0.0479,True,False,False,False,False,True,False,False,True,True,True,False,False
6,7,False,False,5.0,NaN,True,False,False,False,False,False,False,False,False,NaN,NaN,False,be_after_push,8.465517,7.482759,1.1161,False,False,False,False,2025-01-02 01:45:55.924221,1.1652,0.0491,0.0058,False,False,False,False,False,7.482759,1.1595,False,False,1.1103,False,tokyo,0.0434,False,False,True,False,long,0.0434,True,False,False,True,False,False,False,False,False,True,True,False,False
7,8,False,False,3.0,NaN,False,False,False,False,False,True,False,False,False,NaN,NaN,False,no_management,2.658228,2.215190,1.1125,False,False,False,False,2025-01-02 01:45:55.924621,1.1335,0.0210,0.0079,False,False,False,False,True,2.215190,1.1300,False,False,1.1046,False,tokyo,0.0175,False,False,True,False,long,0.0175,False,True,False,False,False,True,False,False,False,True,Tru

In [44]:
full_df.dtypes

trade_uid                                    object
type_3_m1                                      bool
type_3_M                                       bool
numerical_confidence                        float64
outcome                                    category
type_1_m5                                      bool
type_2_m30                                     bool
type_1_m1                                      bool
type_3_h4                                      bool
type_3_w                                       bool
htf_poi_ltf_confirmation                   category
type_2_m15                                     bool
type_3_m15                                     bool
type_2_h4                                      bool
return                                      float64
close_price                                 float64
type_1_m30                                     bool
management_strategy                        category
potential_risk_reward                       float64
initial_risk

In [45]:
!rm -rf output

In [47]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
j.to_markdown(output_dir='output')

INFO:root:Creating initial trades
INFO:root:Adding trade with UID: 1
INFO:root:Adding trade with UID: 2
INFO:root:Adding trade with UID: 3
INFO:root:Adding trade with UID: 4
INFO:root:Adding more trades to the journal
INFO:root:Adding trade with UID: 5
INFO:root:Adding trade with UID: 6
INFO:root:Adding trade with UID: 7
INFO:root:Adding trade with UID: 8
INFO:root:Adding trade with UID: 9
INFO:root:Adding trade with UID: 10
INFO:root:Adding trade with UID: 11
INFO:root:Adding trade with UID: 12
INFO:root:Adding trade with UID: 13
INFO:root:Adding trade with UID: 14
INFO:root:Adding default tags to all trades
INFO:root:Converting journal trades to DataFrame
INFO:root:Added PA tags to DataFrame. Columns: Index(['type_1_m5', 'type_1_m1', 'type_1_m30', 'type_1_h4', 'type_1_m15',
       'risk_reward_ratio', 'type_1_h1', 'type_1_d', 'type_1_M', 'type_1_w',
       'has_type_1', 'has_type_2', 'has_type_3', 'has_time_frame'],
      dtype='object')
INFO:root:DataFrame columns: Index(['type_1_m5

frozenset({'type_1_m5', 'type_2_m30', 'M', 'type_1_m1', 'type_2_m15', 'type_2_h4', 'm15', 'type_1_m30', 'type_2_M', 'type_2_m1', 'm1', 'type_2_w', 'h4', 'm30', 'type_1_h4', 'type_2_m5', 'type_2_h1', 'type_1_m15', 'd', 'type_1_h1', 'type_1_d', 'm5', 'type_1_M', 'w', 'h1', 'type_2_d', 'type_1_w'})


INFO:root:Converting trades to markdown
INFO:root:Files for trades found
INFO:root:Files Found #: 14, Total Trades #: 14max files for trade: 0min files for trade: 0avg files for trade: 0.0
INFO:root:Descriptive Statistics:           0
count  14.0
mean    0.0
std     0.0
min     0.0
25%     0.0
50%     0.0
75%     0.0
max     0.0
INFO:root:Writing index markdown
INFO:root:Calculating simple statistics
INFO:root:Analyzing tag relevance
INFO:root:Finding best single tags and subsets
INFO:root:Finding best single tags
INFO:root:Finding best tag subsets
INFO:root:Finding best tag subsets
INFO:root:Selecting top 10 features using RFE
INFO:root:Dropped 5 rows with NaN in 'outcome' y variable
INFO:root:X features # 42: {'type_3_m1', 'type_3_M', 'htf_poi_ltf_confirmation_False', 'type_2_m30', 'type_1_m5', 'type_1_m1', 'type_3_h4', 'type_3_w', 'htf_poi_ltf_confirmation', 'type_2_m15', 'type_3_m15', 'type_2_h4', 'close_price', 'type_1_m30', 'management_strategy', 'outcome_loss', 'type_3_d', 'type